In [ ]:
-- 1
select name, record
from sports
order by name;

In [ ]:
-- 1 Refactor
select Name, Record 
from Sports order by name;

In [ ]:
-- 2
select distinct s.name
from results r
join sports s
on s.id = r.sportid;

In [ ]:
-- 2 Refactor
select distinct S.name
from Results R
join Sports S on S.id = R.sportid;

In [ ]:
-- 3
select count(distinct peopleid)
from results;

In [ ]:
-- 4
select p.id, p.name 
from (
    select r.peopleid as id
    from results r
    group by r.peopleid
    having count(r.peopleid) >= 20
) as r 
join people p 
on p.id = r.id;

In [ ]:
-- 5
select distinct p.id, p.name, g.description
from sports s
join results r 
on s.id = r.sportid
and s.record = r.result
join people p 
on r.peopleid = p.id 
join gender g 
on p.gender = g.gender;

In [ ]:
-- 6
select s.name, count(distinct r.peopleid) as numathletes
from sports s 
join results r 
on s.record = r.result 
and s.id = r.sportid 
group by s.name;

In [ ]:
-- 7
select p.id, p.name, r.best, r.difference 
from (
    select 
        r.peopleid as id, 
        max(r.result) as best, 
        to_char(s.record-max(r.result), '0D99') as difference
    from results r 
    join sports s
    on s.name like 'Triple Jump'
    and s.id = r.sportid
    group by r.peopleid, s.record
    having count(r.peopleid) >= 20
) as r
join people p 
on r.id = p.id;

In [ ]:
-- 8
select distinct p.id, p.name, g.description 
from (
    select c.id from competitions c
    where c.place like 'Hvide Sande'
    and extract(year from c.held) = 2009
) as c
join results r 
on c.id = r.competitionid
join people p 
on r.peopleid = p.id
join gender g 
on g.gender = p.gender;

In [ ]:
-- 9
select p.name, g.description
from people p
join gender g 
on g.gender = p.gender
and p.name like '% J%sen'; 

In [ ]:
-- 10
select 
    p.name, 
    s.name, 
    case when r.result is null then to_char(0,'999%')
    else to_char(100*(r.result/s.record),'999%') end as percentage
from sports s 
join results r 
on s.id = r.sportid
join people p 
on r.peopleid = p.id;

In [ ]:
-- 11
select count(distinct r.peopleid)
from results r
where result is null;

In [ ]:
-- 12
select s.id, s.name, to_char(max(r.result), '999D99') as maxres 
from sports s 
join results r 
on s.id = r.sportid
group by s.id, s.name
order by s.id;

In [ ]:
-- 13
select p.id, p.name, r.records
from (
    select r.peopleid as id, count(s.id) as records
    from results r
    join sports s 
    on r.result = s.record
    and s.id = r.sportid
    group by r.peopleid
    having count(distinct s.id) >= 2
) as r
join people p
on r.id = p.id;

In [ ]:
-- 14
drop view if exists max_results;
create view max_results as
select sportid, max(result) as result
from results
where result is not null
group by sportid;

select 
    r.peopleid,
    p.name,
    p.height,
    r.result,
    s.name, 
    case when r.result = s.record 
        then 'yes'
        else 'no' 
    end as record

from max_results mr
join sports s 
on mr.sportid = s.id
join results r
on mr.result = r.result
and mr.sportid = r.sportid
join people p 
on p.id = r.peopleid 
group by (r.peopleid, p.name, p.height, r.result, s.name, s.record);

In [ ]:
-- 15
select p.id, p.name
from results r
right outer join people p
on r.peopleid = p.id
where competitionid is null;

In [ ]:
-- 16
select distinct id, name from 
(
    select p.id, p.name
    from sports s 
    join results r
    on s.name like 'High Jump'
    and s.id = r.sportid
    and s.record = r.result
    join people p
    on r.peopleid = p.id
) record_holders
union
( 
    select p.id, p.name
    from competitions c
    join results r
    on c.id = r.competitionid
    and extract(year from c.held) = 2002
    AND extract(MONTH FROM c.held) = 6
    join people p
    on r.peopleid = p.id 
);

In [ ]:
-- 17
select ppl.id, ppl.name from
(
    select peopleid as id
    from results r
    group by r.peopleid
    having count(distinct r.sportid) = 1
) p
join results r
on r.peopleid = p.id
and r.result = (select record from sports where id = r.sportid)
join people ppl
on ppl.id = p.id;

In [ ]:
-- 18
select count(*) from (
    select r.peopleid
    from results r 
    join competitions c
    on r.competitionid = c.id
    group by r.peopleid
    having count(distinct c.place) >= 10
) p;

In [ ]:
-- 19
select p.id, p.name from
( 
    select r.peopleid
    from sports s
    join results r 
    on s.record = r.result
    group by r.peopleid
    having count(distinct s.id) = (select count(distinct id) from sports)
) r
join people p
on r.peopleid = p.id;

In [ ]:
-- 20
select S.id, S.name, S.record, min(R.result)
from Sports S
    join Results R on S.id = R.sportid
    join Competitions C on R.competitionid = C.id
group by S.id
having count(distinct C.place) = (
    select count(distinct place)
    from Competitions
);